In [ ]:
from tqdm.notebook import tqdm
from math import sqrt,erf
from multiprocessing import Pool
from itertools import product, repeat
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib as mpl
import numpy as np
from varname import nameof
from qutip import *
from matplotlib.cm import get_cmap
import colorcet as cc
import subprocess
import os, shutil
import time
import functools
from scipy import special
import scipy
from IPython.display import display, clear_output


plt.style.use('ggplot')
markers = ["o", "X", "P", "p", "*"]
cols = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{braket}\usepackage{nicefrac}')
plt.rcParams.update({'font.size': 30,
                     'figure.figsize': (11,7),
                     'axes.facecolor': 'white',
                     'axes.edgecolor': 'lightgray',
                     "figure.autolayout": 'True',
                     'axes.xmargin': 0.03,
                     'axes.ymargin': 0.05,
                     'axes.grid': False,
                     'axes.linewidth': 5,
                     'lines.markersize': 15,
                     'text.usetex': True,
                     'lines.linewidth': 8,
                     "legend.frameon": True,
                     "legend.framealpha": 0.7,
                     "legend.handletextpad": 1,
                     "legend.edgecolor": "black",
                     "legend.handlelength": 1,
                     "legend.labelspacing": 0,
                     "legend.columnspacing": 1,
                     "legend.fontsize": 35,
                    })
linestyles = ["-", "--", ":"]
bbox = dict(boxstyle="round", facecolor="white")

D0 = 1
deltaD = 0.005
plt.plot([], [])
plt.show()
clear_output()

In [ ]:
def get_RG_flow(J0_by_D0, Ub_by_J=0, plot=False):
    """ Returns the flow of couplings in the form of two ndarrays J and D.
    Each ndarray is in ascending order of the bandwidth. """
    J0 = D0 * J0_by_D0
    Ub = - Ub_by_J * J0
    omega = -D0/2
    
    ### initialise arrays with UV values
    D = [D0]
    J = [J0]
    
    ### apply URG transformations until bandwith vanishes
    ### or J reduces to zero.
    while D[-1] >= deltaD and J[-1] >= 0:
        
        ### URG equation
        deltaJ = - J[-1] * (J[-1] + 4 * Ub) / (omega - D[-1]/2 + J[-1]/4) * deltaD
        
        ### Check if denominator has changed sign, 
        ### then append renormalised values to array
        if (omega - (D[-1] - deltaD)/2 + (J[-1] + deltaJ)/4) * (omega - D0/2 + J0/4) > 0:
            D.append(D[-1] - deltaD)
            J.append(J[-1] + deltaJ)
        else:
            break
    
    ### plot values
    if plot:
        plt.plot(np.array(D)/D0, np.array(J)/J0, marker="o")
        plt.xlabel(r"$D/D_0$")
        plt.ylabel(r"$J/J_0$")
    print (len(J), D0/deltaD)
    return np.flip(D), np.flip(J)

_ = get_RG_flow(0.35, Ub_by_J=0, plot=True)
# print (D[0]/deltaD, J[0], D[0])

In [ ]:
from  concurrent.futures import ThreadPoolExecutor, as_completed

def get_renorm_per_comb(args_arr):
    target_combs = []
    target_renorm = []
    for args in args_arr:
        source_comb, q_pos, alpha, coeff = args
        # for source_comb in tqdm(combinations, total=len(combinations), leave=False, disable=False):
        # ### loop over spins of IOM, particle/hole sectors as well as cloud members
        for beta, k_pos in product([1, -1], range(1, q_pos - 1, 2)):
            beta_pos = int((1 - beta) / 2)
            betabar_pos = int(1 - beta_pos)
            if  source_comb[k_pos + beta_pos] == 1 - source_comb[q_pos + beta_pos]:
                target_comb = np.copy(source_comb)
                target_comb[k_pos + beta_pos] = 1 - source_comb[k_pos + beta_pos]
                target_comb[q_pos + beta_pos] = 1 - source_comb[q_pos + beta_pos]
                target_combs.append(target_comb)
                target_renorm.append(alpha * beta * (source_comb[0] - 0.5) * coeff)
            if  source_comb[k_pos + betabar_pos] == 1 - source_comb[q_pos + beta_pos] and source_comb[0] == 1 - abs((1 - beta)/2 - source_comb[q_pos + beta_pos]):
                target_comb = np.copy(source_comb)
                target_comb[k_pos + betabar_pos] = 1 - source_comb[k_pos + betabar_pos]
                target_comb[q_pos + beta_pos] = 1 - source_comb[q_pos + beta_pos]
                target_comb[0] = 1 - source_comb[0]
                target_combs.append(target_comb)
                target_renorm.append(alpha * coeff)
        return target_combs, target_renorm


def get_renorm(coefficients, combinations, savepath, total_num, alpha):
    """ Calculates and returns the renormalised coefficients at a given RG step. """
    q_pos = total_num - 2 ### position of the nearest IOM
    try:
        ### uncomment the next line if you don't want to read from saved data
        raise ValueError('A very bad thing happened.')
        
        ### read from saved data, if available
        new_coefficients = np.load(savepath + "-coeff.npz")['arr_0']
        new_combinations = np.load(savepath + "-combs.npz")['arr_0']
    except:
        ### if saved data is not available, run this block
        
        new_coefficients = np.copy(coefficients)
        combinations = np.array([list(c) + [1, 1] for c in combinations])
        new_combinations = np.copy(combinations)
        args = [[source_comb, q_pos, alpha, coefficients[(combinations == source_comb).all(axis=1).nonzero()][0]] for source_comb in combinations]
        target_combs, target_renorm = Pool().apply_async(get_renorm_per_comb, (args, )).get()
        for tcomb,trenorm in zip(target_combs, target_renorm):
            if list(tcomb) not in new_combinations.tolist():
                new_combinations = np.vstack([new_combinations, tcomb])
                new_coefficients = np.append(new_coefficients, 0)
            new_coefficients[(new_combinations == tcomb).all(axis=1).nonzero()] += trenorm

        ### normalise the final coefficients
        new_coefficients /= np.linalg.norm(new_coefficients)
        
        ### Save (with compression!)
        np.savez_compressed(savepath + "-coeff.npz", new_coefficients)
        np.savez_compressed(savepath + "-combs.npz", new_combinations)
    return new_coefficients, new_combinations

def get_init_weight(args):
    comb, total_num, c_all, Xgs = args
    classicalState = tensor([basis(2, 1 - int(comb[0]))] + [basis(2,0)] * (total_num - 1))
    for i,occ in enumerate(comb):
        if i == 0:
            continue
        classicalState = [1, tensor(identity(2), c_all[i - 1].dag())][int(occ)] * classicalState
    weight = np.real(Xgs.dag() * classicalState)[0][0]
    if np.round(weight, 5) != 0:
        return comb, weight
    else: 
        return
    
def init_wavefunction(J, D, num_in):
    """ Generates the initial wavefunction at the fixed point.
    Returns the coefficients associated with it and the list
    of all possible occupancies for each of the members.
    No IOMS are taken into account at this point."""
    
    total_num = 1 + 2 * num_in
    c_all = [tensor([sigmaz()]*(i) + [destroy(2)] + [identity(2)]*(total_num - 1 - i - 1)) for i in range(total_num - 1)]
    num_kup = [c.dag() * c for c in c_all[0::2]]
    num_kdn = [c.dag() * c for c in c_all[1::2]]
    id_bath = tensor([identity(2)] * (total_num - 1))
    c0up = sum(c_all[::2]) / (0.5 * (total_num - 1))
    c0dn = sum(c_all[1::2]) / (0.5 * (total_num - 1))
    sigma0z = c0up.dag() * c0up - c0dn.dag() * c0dn
    Ek = np.linspace(-D, -deltaD, num_in)
    H = sum([Ek[i] * tensor(identity(2), (num_kup[i] + num_kdn[i])) for i in range(len(num_kup))]) \
            + J * (0.25 * tensor(sigmaz(), sigma0z)
            + 0.5 * tensor(sigmap(), c0dn.dag() * c0up)
            + 0.5 * tensor(sigmam(), c0up.dag() * c0dn)
                  )
    H_scipy = np.array(H)
    E, v = scipy.sparse.linalg.eigsh(H_scipy, k=1, which='SA')
    Xgs = v[:,0]
    Xgs_q = Qobj(Xgs, dims=[[2] * total_num, [1] * total_num])
    assert np.real(Xgs_q.dag() * (tensor(sigmaz(), id_bath) + tensor(identity(2), sigma0z)) * Xgs_q)[0][0] == 0
        
    args = [[comb, total_num, c_all, Xgs_q] for comb in product([1,0], repeat=total_num)]
    results = list(tqdm(Pool().imap(get_init_weight, args), total=2**total_num))
    combinations = np.array([r[0] for r in results if r!= None])
    coefficients = np.array([r[1] for r in results if r!= None])
    return coefficients/np.linalg.norm(coefficients), combinations


def get_tensorRG(J0_by_D0, num_in, num_out, computables):

    D, J = get_RG_flow(J0_by_D0)
    # num_in = round(D[0]/deltaD)
    print ("Starting with", num_in,"k-states in the bath.") 
    omega = -D0/2
    
    ### ensure that there are enugh RG steps in J and D
    ### to recouple all IOMS.
    assert len(J) >= num_out + 1
    
    ### obtain the initial coefficients and set of occupancies
    savepath = "./MERGCoeffs/{:.0f},{:.0f},{:.10f},{:.10f},{:.10f},{:.10f}".format(round(D[0]/deltaD), 1 + 2 * num_in, J0_by_D0 * D0, D0, deltaD, omega)
    try:
        raise ValueError('A very bad thing happened.')
        coefficients = np.load(savepath + "-coeff.npz")['arr_0']
        combinations = np.load(savepath + "-combs.npz")['arr_0']
    except:
        coefficients, combinations = init_wavefunction(J[0], D[0], num_in)
        os.makedirs("./MERGCoeffs", exist_ok=True)
        np.savez_compressed(savepath + "-coeff.npz", coefficients)
        np.savez_compressed(savepath + "-combs.npz", combinations)
    ### save the initial set of coefficients and store the savepath
    ### in the list of savepaths that will be finally returned.
    coefficients_arr = [coefficients]
    savepaths = []
    savepaths.append(savepath)
    
    computations = {computable: [] for computable, members, _, _ in computables}
    ### initial the next set of coefficients and loop through 
    ### the RG flow to generate the tensor network RG.
    
    new_combinations = np.copy(combinations)
    for Ji, Di in tqdm(zip(J[1: num_out + 1], D[1: num_out + 1]), total=num_out, desc="Applying inverse unitaries"):
        total_num = 1 + 2 * num_in + 2
        alpha = Ji / (omega - Di/2 + Ji/4)
        savepath = "./MERGCoeffs/{:.0f},{:.0f},{:.10f},{:.10f},{:.10f},{:.10f}".format(round(D[0]/deltaD), total_num, Ji, Di, deltaD, omega)
        
        ### get the next set of coefficients and append the savepath to access them later.
        coefficients, combinations = get_renorm(coefficients, combinations, savepath, total_num, alpha)
        savepaths.append(savepath)
        
        ### increase the number of entangled members.
        num_in += 1
            
    for computable, members, ylabel, savename in computables:
        if computable == "VNE":
            args = [[savepath, members] for savepath in savepaths]
            computations[computable] = list(tqdm(Pool().imap(get_VNE_perstep, args), total=len(savepaths)))
            plt.scatter(range(round(D[0]/deltaD), round(D[0]/deltaD) + num_out + 1), computations[computable])
            plt.xlabel(r"no. of $k-$states in the c-bath")
            plt.ylabel(ylabel)
            plt.savefig(savename, bbox_inches='tight')
            plt.show()
        if computable == "I2":
            assert len(members) == 2
            args1 = [[savepath, members[0]] for savepath in savepaths]
            S1 = list(tqdm(Pool().imap(get_VNE_perstep, args1), total=len(savepaths)))
            args2 = [[savepath, members[1]] for savepath in savepaths]
            S2 = list(tqdm(Pool().imap(get_VNE_perstep, args2), total=len(savepaths)))
            args12 = [[savepath, list(members[0]) + list(members[1])] for savepath in savepaths]
            S12 = list(tqdm(Pool().imap(get_VNE_perstep, args12), total=len(savepaths)))
            computations[computable] = np.array(S1) + np.array(S2) - np.array(S12)
            plt.scatter(range(round(D[0]/deltaD), round(D[0]/deltaD) + num_out + 1), computations[computable])
            plt.xlabel(r"no. of $k-$states in the c-bath")
            plt.ylabel(ylabel)
            plt.savefig(savename, bbox_inches='tight')
            plt.show()
        if computable == "Qtemp":
            args = [[savepath, members] for savepath in savepaths]
            computations[computable] = list(tqdm(Pool().imap(get_Qtemp_perstep, args), total=len(savepaths)))
            plt.scatter(range(round(D[0]/deltaD), round(D[0]/deltaD) + num_out + 1), computations[computable])
            plt.xlabel(r"no. of $k-$states in the c-bath")
            plt.ylabel(ylabel)
            plt.savefig(savename, bbox_inches='tight')
            plt.show()
            
    return computations


def get_VNE_perstep(args):
    """Returns the VNE at a given RG step."""

    savepath, members = args
    ### Access the coefficients data for the given RG step.
    coefficients = np.load(savepath + "-coeff.npz")['arr_0']
    combinations = np.load(savepath + "-combs.npz")['arr_0']
    ### Create the reduced density matrix by adding 
    ### the pertinent coefficients.
    rho_reduced = np.zeros([2**len(members)] * 2)
    members_combs = [list(comb) for comb in product([1,0], repeat=len(members))]
    for i, comb_i in enumerate(combinations):
        index_i = members_combs.index(list(comb_i[members]))
        for index_j, member_comb in enumerate(members_combs):
            comb_j = np.copy(comb_i)
            comb_j[members] = member_comb
            if list(comb_j) in [list(item) for item in combinations]:
                # print (comb_j)
                j = [list(item) for item in combinations].index(list(comb_j))
                rho_reduced[index_i, index_j] = coefficients[i] * coefficients[j]

    rho_red_qutip = Qobj(rho_reduced, dims = [[2] * len(members)] * 2)
    SEE = entropy_vn(rho_red_qutip)
    
    return SEE


def get_Qtemp_perstep(args):
    """Returns the VNE at a given RG step."""

    savepath, member = args
    ### Access the coefficients data for the given RG step.
    coefficients = np.load(savepath + "-coeff.npz")['arr_0']
    combinations = np.load(savepath + "-combs.npz")['arr_0']

    ### Create the reduced density matrix by adding 
    ### the pertinent coefficients.
    rho_reduced = np.zeros([2,2])
    
    
    rho_reduced[0][0] = sum(coefficients[combinations[:,member] == 0]**2)
    rho_reduced[1][1] = sum(coefficients[combinations[:,member] == 1]**2)

    ### Diagonalise reduced DM and return VNE.
    
    eigvals = np.linalg.eigvalsh(rho_reduced)

    return np.log(abs(rho_reduced[1][1] / rho_reduced[0][0]))

In [ ]:
computations = get_tensorRG(0.35, 4, 20, [['I2', [[0], [1, 2]], r"$I_2(d:k_1)$", "I2_dk.pdf"], ['I2', [[1, 2], [3, 4]], r"$I_2(k_1:k_2)$", "I2_kk.pdf"]])